In [26]:
from typing import List, Optional

In [2]:
# Dataset
DATA_FILE = "/home/taot/data/huggingface/my-neural-network-data/gpt-from-scratch/tiny_shakespeare.txt"
with open(DATA_FILE, "r") as fp:
    text = fp.read()
print(len(text))

1115394


In [3]:
# Character-based tokenize
def get_vocab(data: str) -> List[str]:
    vocab = sorted(set(text))
    print(f"vocab_size = {len(vocab)}")
    print(f"vocab = {''.join(vocab)}")
    return vocab

vocab = get_vocab(text)

vocab_size = 65
vocab = 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [4]:
stoi = {c:i for i, c in enumerate(vocab)}
itos = {i:c for i, c in enumerate(vocab)}

def encode(s: str) -> List[str]:
    return [stoi[c] for c in s]

def decode(ids: List[int]) -> str:
    return ''.join([itos[i] for i in ids])

In [5]:
ids = encode("hello")
print(ids)
print(decode(ids))

[46, 43, 50, 50, 53]
hello


In [6]:
# Convert data to tensor
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.size())
print(data[:100])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [7]:
# Split the data into train and validation
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[0:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"input = {context}, target = {target}")

input = tensor([18]), target = 47
input = tensor([18, 47]), target = 56
input = tensor([18, 47, 56]), target = 57
input = tensor([18, 47, 56, 57]), target = 58
input = tensor([18, 47, 56, 57, 58]), target = 1
input = tensor([18, 47, 56, 57, 58,  1]), target = 15
input = tensor([18, 47, 56, 57, 58,  1, 15]), target = 47
input = tensor([18, 47, 56, 57, 58,  1, 15, 47]), target = 58


In [10]:
torch.randint(3, (2, 3))

tensor([[1, 1, 2],
        [1, 0, 2]])

In [11]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split: str):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [12]:
xb, yb = get_batch("train")

In [54]:
# Bigram language model
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch import Tensor

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size: int):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx: Tensor, targets: Optional[Tensor] = None):
        # idx and targets are both (B,T). B is batch_size, T is time (block_size which is 8), C is classes
        logits = self.token_embedding_table(idx)  # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss   # (B,T,C)

    def generate(self, idx: Tensor, max_new_tokens: int) -> Tensor:
        # idx: (B, T)
        for _ in range(max_new_tokens):
            logits, _ = self(idx)   # (B, T, C)
            logits = logits[:, -1, :]   # (B, 1, C)
            
            probs = torch.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            
            idx = torch.cat([idx, idx_next], dim=1)   # (B, T+1)

        return idx            

In [55]:
vocab_size = len(vocab)
m = BigramLanguageModel(vocab_size)
# logits, loss = m(xb, yb)
# print(f"xb.size() = {xb.size()}")
# print(f"logits.size() = {logits.size()}")
# print(f"loss.size() = {loss.size()}")
# print(f"loss = {loss}")

decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=50)[0].tolist())

"\nSr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLER"

In [ ]:
# Train the model
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)
batch_size = 32
for step in range(1000):
    xb, yb = get_batch("train")
    targets, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"step = {step}, loss = {loss.item()}")

In [63]:
decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist())

"\n\nD tt a wisoouss we hend thawim I fif RDIVI tharf, my\nHMAng.\nIfomer\n\n\nARFiaulfitome nakies w,\nWh t, reas ouckisouleerolutow m:\n\nWherond bale g d bres hof wit'st Wh'the.\nLishor e abe go g ad gang tamy s widir d Ind ase.\nTLI:\nTHAN:\nAnagrt to ss\nThy Rit aler,\nguscooo tshoupar st athay tt\nWAwe tor,\nOFo lyourcal oope bllee?\nWhu we o whavetous! pers farthethulos Hee omyou Vins VInd bast:\n\nD:\n\nAbishiethe loul?\n\nS:\nKELI, berisho llle;\nObr ps 'ded med, d in thathret-vetomy ge your may lil'diteranoneo MAS"

# Mathematical trick in self-attention